In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [19]:
def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise


def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """

    n_bits = len(word)
    
    pw = qml.pauli.PauliWord({i:s for i,s in enumerate(word)})
    rho = (np.eye(2**n_bits) + pw.to_mat(range(n_bits))) / 2**n_bits
    
    dev = qml.device('default.mixed', wires=n_bits)
    @qml.qnode(dev)
    def pauli_circuit(rho, sz, lmbda):
        qml.QubitDensityMatrix(rho, wires=range(sz))
        for i in range(sz):
            qml.DepolarizingChannel(lmbda,wires=i)
            
        return qml.density_matrix(wires=range(sz))
    
    density_matrix = pauli_circuit(rho, n_bits, lmbda)
    return density_matrix

# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """


    # Put your code here #
    sz = len(word)
    mixed_state = np.eye(2**sz)/2**sz
    pauli_state = noisy_Pauli_density(word, lmbda)
    dist = abs_dist(pauli_state, mixed_state)
    return 0.5*np.trace(dist)


def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """


    # Put your code here #
    p = word_dist(word)
    return (1-lmbda)**p - maxmix_trace_dist(word, lmbda)

In [14]:
word = 'XXY'

pd = noisy_Pauli_density('XXY',0.1)
mix = np.eye(2**3)/2**3

dist = abs_dist(pd,mix)
p = word_dist(word)

In [16]:
maxmix_trace_dist(word, 0.1)

(0.32548148148148154+0j)

In [20]:
bound_verifier(word, 0.)

(0.5000000000000002+0j)

In [6]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    word, lmbda = json.loads(test_case_input)
    output = np.real(bound_verifier(word, lmbda))

    return str(output)


def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your trace distance isn't quite right!"

In [21]:
test_cases = [['["XXI", 0.7]', '0.0877777777777777'], ['["XXIZ", 0.1]', '0.4035185185185055'], ['["YIZ", 0.3]', '0.30999999999999284'], ['["ZZZZZZZXXX", 0.1]', '0.22914458207245006']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '["XXI", 0.7]'...
Correct!
Running test case 1 with input '["XXIZ", 0.1]'...
Correct!
Running test case 2 with input '["YIZ", 0.3]'...
Correct!
Running test case 3 with input '["ZZZZZZZXXX", 0.1]'...
Correct!
